In [121]:
import numpy as np
import pandas as pd
from collections import defaultdict
import heapq

In [81]:
df = pd.read_csv('kobe.csv')
df.head()

,action_type,combined_shot_type,game_id,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,game_date,matchup,opponent
0,Jump Shot,Jump Shot,20000012,10,1,0,Jan-00,27,18,0,2PT Field Goal,Right Side(R),Mid-Range,2000/10/31,LAL @ POR,POR
1,Jump Shot,Jump Shot,20000012,10,1,0,Jan-00,22,15,0,2PT Field Goal,Left Side(L),Mid-Range,2000/10/31,LAL @ POR,POR
2,Jump Shot,Jump Shot,20000012,7,1,0,Jan-00,45,16,0,2PT Field Goal,Left Side Center(LC),Mid-Range,2000/10/31,LAL @ POR,POR
3,Jump Shot,Jump Shot,20000012,6,1,0,Jan-00,52,22,0,2PT Field Goal,Right Side Center(RC),Mid-Range,2000/10/31,LAL @ POR,POR
4,Driving Dunk Shot,Dunk,20000012,6,2,0,Jan-00,19,0,0,2PT Field Goal,Center(C),Restricted Area,2000/10/31,LAL @ POR,POR


In [22]:
def question1():
    vs_HOU = df[df['opponent']=='HOU']
    vs_HOU_2pt = vs_HOU[vs_HOU['shot_type']=='2PT Field Goal']
    vs_HOU_3pt = vs_HOU[vs_HOU['shot_type']=='3PT Field Goal']

    n_row, n_col= vs_HOU_2pt.shape
    shot_made_series = vs_HOU_2pt['shot_made_flag']
    rate_2pt = shot_made_series.sum()/n_row

    n_row, n_col= vs_HOU_3pt.shape
    shot_made_series = vs_HOU_3pt['shot_made_flag']
    rate_3pt = shot_made_series.sum()/n_row
    
    print("Kobe 對戰火箭隊(HOU)的平均兩分球與三分球命中率:")
    print("average 2PT FG% : "+str(rate_2pt))
    print("average 3PT FG% : "+str(rate_3pt))


In [115]:
question1()

Kobe 對戰火箭隊(HOU)的平均兩分球與三分球命中率:
average 2PT FG% : 0.38814814814814813
average 3PT FG% : 0.26582278481012656


In [134]:
def question2():
    df = pd.read_csv('kobe.csv')

    opponents = df.drop_duplicates(subset='opponent', keep='first')['opponent']
    opponents = opponents.tolist()

    oppo_pt = {}

    for oppo in opponents:
        data = df[df['opponent']==oppo]
        game_cnt = len(data.drop_duplicates(subset='game_id', keep='first', inplace=False))
        point_2pt = 2*len(data[(data.shot_type == '2PT Field Goal') & (data['shot_made_flag']==1)])
        point_3pt = 3*len(data[(data.shot_type == '3PT Field Goal') & (data['shot_made_flag']==1)])
        point = (point_2pt+point_3pt)/game_cnt
        oppo_pt[oppo] = point

    print("使得 Kobe 平均得分最低的前五支球隊")
    top5 = heapq.nsmallest(5, zip(oppo_pt.values(), oppo_pt.keys()))
    for i in range(5):
        print (top5[i])


In [135]:
question2()

使得 Kobe 平均得分最低的前五支球隊
(11.166666666666666, 'VAN')
(12.666666666666666, 'BKN')
(14.025641025641026, 'IND')
(14.142857142857142, 'UTA')
(14.178571428571429, 'NJN')


In [152]:
def question3():
    df = pd.read_csv('kobe.csv')

    game_ids = df.drop_duplicates(subset='game_id', keep='first')['game_id']
    game_ids = game_ids.tolist()

    py_q4 = df[(df['playoffs']==1) & (df['period']==4)]
    py_q4_2m = py_q4[py_q4['minutes_remaining']==2]
    py_q4_1m = py_q4[py_q4['minutes_remaining']==1]
    py_q4_0m = py_q4[py_q4['minutes_remaining']==0]

    py_q4_3m = pd.concat([py_q4_2m, py_q4_1m, py_q4_0m], ignore_index=True)

    pt_game = {}
    for _id in game_ids:
        shots = py_q4_3m[py_q4_3m['game_id']==_id]
        shot_made_2pt = shots[(shots['shot_type']=='2PT Field Goal')&(shots['shot_made_flag']==1)]
        shot_made_3pt = shots[(shots['shot_type']=='3PT Field Goal')&(shots['shot_made_flag']==1)]
        point = 2*len(shot_made_2pt)+3*len(shot_made_3pt)
        pt_game[_id] = point
    top5 = heapq.nlargest(5, zip(pt_game.values(), pt_game.keys()))
    print('在季後賽最後 3 分鐘內得分最高的前五場球賽')
    print('(得分, game id)')
    for i in top5:
        print(i)

In [153]:
question3()

在季後賽最後 3 分鐘內得分最高的前五場球賽
(得分, game id)
(9, 49900074)
(8, 40100234)
(7, 49900024)
(7, 40700402)
(6, 49600024)


In [179]:
def question4():
    df = pd.read_csv('kobe.csv')
    df = df.sort_values(by='game_date')
    filted = df[(df['playoffs']==1)&(df['minutes_remaining']==0)&(df['period']==4)&(df['action_type']=='Jump Shot')]
    filted.reset_index(drop=True, inplace=True)

    rate_season = {}
    season_remain = filted.loc[0].season
    game_cnt = 0
    shot_made_cnt = 0 
    for i in range(len(filted)):
        if filted.loc[i].season!=season_remain:
            rate_season[season_remain] = float(shot_made_cnt)/game_cnt
            game_cnt = 1
            shot_made_cnt = filted.loc[i].shot_made_flag
            season_remain = filted.loc[i].season

        else:
            game_cnt+=1
            shot_made_cnt+=filted.loc[i].shot_made_flag
            
    print('各球季季後賽中，比賽最後 1 分鐘內的 Jump Shot 命中率，按球季先後排序')
    print('season : rate')
    cnt = 0
    rate = 0
    for _season in rate_season.keys():
        cnt+=1
        if cnt<5:
            print(_season+" : "+str(rate_season[_season]))
        else:
            rate+=rate_season[_season]
    print('2000-01 : '+str(rate/10))
    

In [180]:
question4()

各球季季後賽中，比賽最後 1 分鐘內的 Jump Shot 命中率，按球季先後排序
season : rate
1996-97 : 0.4
1997-98 : 0.0
1998-99 : 0.14285714285714285
1999-00 : 0.1
2000-01 : 0.1976190476190476


In [215]:
def question5():
    df = pd.read_csv('kobe.csv')
    game_ids = df.drop_duplicates(subset='game_id', keep='first', inplace=False)['game_id']
    game_ids = game_ids.tolist()
    rate_date = []

    for _id in game_ids:
        data = df[df['game_id']==_id].reset_index(drop=True, inplace=False)
        date = data.loc[0].game_date
        rate = float(data['shot_made_flag'].sum()) / len(data)
        rate_date.append((rate, date))


    cnt_date = {}
    cnt = 0
    new = 1
    for _iter in rate_date:
        if _iter[0]>=0.33:
            cnt+=1
            if new==1:
                start_date = _iter[1]
                new = 0
            else:
                end_date = _iter[1]
        else:
            if new==0:   
                cnt_date[start_date+'~'+end_date] = cnt
                new = 1
                cnt = 0
    top3 = heapq.nlargest(3, zip(cnt_date.values(), cnt_date.keys()))
    print('「得分命中率 33%以上的最長連續場數」前 3 名，列出場數以及起訖日期')
    print('場數 : 起訖日期')
    for _iter in top3:
        print(str(_iter[0])+' : '+_iter[1])

In [216]:
question5()

「得分命中率 33%以上的最長連續場數」前 3 名，列出場數以及起訖日期
場數 : 起訖日期
22 : 2001/4/10~2001/12/7
19 : 2000/2/1~2000/3/12
16 : 2002/1/14~2002/2/17


In [ ]:
def question6():
    